# Setup

In [27]:
# @markdown #### Caso escolheu o tipo de recuperação drive/ambos e não tenha os arquivos, faça um atalho desse drive:
link_do_drive = "https://drive.google.com/drive/folders/1CzaSibzU51qCF0WYFyieMqZAsn-tNoia?usp=drive_link" #@param {type:"string"}
path_atalho_drive = "/content/drive/MyDrive/Projetos Do GitHub/temp" #@param {type:"string"}
path_download = "temp" #@param {type:"string"}

In [28]:
import os
import zipfile

if os.path.exists('python_download_file_drive'):
  print("Pasta python_download_file_drive já existe.")
  !cd python_download_file_drive && git pull
else:
  print("Pasta python_download_file_drive não existe. Clonando...")
  !git clone https://github.com/KauaHenSilva/python_download_file_drive
%pip install --upgrade --quiet -r python_download_file_drive/requirements.txt

Pasta python_download_file_drive já existe.
Already up to date.


In [29]:
try:
    from google.colab import drive
    from google.colab import auth
    from googleapiclient.discovery import build
    auth.authenticate_user()

    !python "python_download_file_drive/main.py" {link_do_drive} {path_download} # ALERTA: Caso o drive bote limite de download comente essa linha

    drive.mount('/content/drive')
    if not os.path.exists(path_atalho_drive):
      print("[INFO] Atalho no drive não encontrado.")

except:

    if os.path.exists(f"{path_download}.zip"):
      print(f"Descompactando {path_download}.zip")
      !rm -rf {path_download}
      with zipfile.ZipFile(f"{path_download}.zip", 'r') as zip_ref:
        zip_ref.extractall(f"{path_download}_temp")
        !mv {path_download}_temp/* {path_download}
        !rm -rf {path_download}_temp
        !rm -rf {path_download}.zip

    if not os.path.exists(path_download):
      raise Exception(f"Você não está nos altenticado nos servidores do google colab. Para prosseguir, faça o download dos arquivos que estão presente no drive e tente novamente. O link do drive {link_do_drive}. A pasta deve está dentro de /content/")

In [30]:
def get_referencia_valida(path, path_atalho_drive, file_init):
    path_download = os.path.join(file_init, path)
    path_drive = os.path.join(path_atalho_drive, path)

    bool_download = os.path.exists(path_download)
    bool_drive = os.path.exists(path_drive)

    if not bool_download and not bool_drive:
        raise Exception(f"Arquivo não encontrado em nenhum dos caminhos (Download ou Drive). Caminhos: {path_download}, {path_drive}")

    if not bool_drive and bool_download:
        print("[INFO] Arquivo encontrado no download, mas não no drive.")
        return path_download

    if bool_drive and not bool_download:
        print("[INFO] Arquivo encontrado no drive, mas não no download.")
        return path_drive

    return path_download

# Importing required libraries

In [31]:
!cp -r "{get_referencia_valida('rbm.py', path_atalho_drive, path_download)}" .

[INFO] Arquivo encontrado no download, mas não no drive.


In [36]:
import numpy as np
import rbm

# Database

In [39]:
base = np.array([[1, 1, 1, 0, 0, 0], [1, 0, 1, 0, 0, 0], [1, 1, 1, 0, 0, 0], [0, 0, 1, 1, 1, 1], [0, 0, 1, 1, 0, 1], [0, 0, 1, 1, 0, 1]])
filmes = ["A bruxa", "Invocação do mal", "O chamado", "Se beber não case", "Gente grande", "American pie"]
base.shape, len(filmes)

((6, 6), 6)

# train

In [41]:
rbm_class = rbm.RBM(num_visible=6, num_hidden=2)
rbm_class.train(base, max_epochs=5000)

Epoch 0: error is 9.097916143024275
Epoch 1: error is 8.864120290429142
Epoch 2: error is 8.6848264475452
Epoch 3: error is 8.598575867795333
Epoch 4: error is 8.458120508370424
Epoch 5: error is 8.292227206157428
Epoch 6: error is 8.246745649899484
Epoch 7: error is 8.158184224833683
Epoch 8: error is 8.022205093378592
Epoch 9: error is 8.008735284901132
Epoch 10: error is 7.854578570760047
Epoch 11: error is 7.771663258941034
Epoch 12: error is 7.747924868443585
Epoch 13: error is 7.5822899053200485
Epoch 14: error is 7.572024098469321
Epoch 15: error is 7.560444904812152
Epoch 16: error is 7.590833829330778
Epoch 17: error is 7.3502941641459625
Epoch 18: error is 7.365823354529322
Epoch 19: error is 7.172715556849053
Epoch 20: error is 7.174734457359599
Epoch 21: error is 7.339480768986207
Epoch 22: error is 7.40105123010516
Epoch 23: error is 7.187753396544871
Epoch 24: error is 7.205128848724435
Epoch 25: error is 6.99159587767608
Epoch 26: error is 6.972703350202854
Epoch 27: err

In [45]:
rbm_class.weights, rbm_class.weights.shape

(array([[ 2.47022897,  2.20720333,  0.37985905],
        [-0.98836825, -3.38900061,  7.93792567],
        [ 0.29345381, -7.20650311,  3.40332288],
        [ 4.94807809,  3.78556404,  1.88328958],
        [ 0.93779797,  3.63472334, -8.23740955],
        [-1.31687274,  0.61174665, -4.55095718],
        [ 0.94291367,  3.63080835, -8.23993484]]),
 (7, 3))

# Simulating

In [60]:
camada_escondida = np.array([[1, 0]])
def avaliar_usuario(usuario):
    probabilidade_hidden = rbm_class.run_visible(usuario)
    recomendacao = rbm_class.run_hidden(probabilidade_hidden)
    return recomendacao

In [65]:
usuario1 = np.array([[1, 1, 0, 0, 0, 0]])
usuario2 = np.array([[0, 0, 0, 1, 1, 0]])
usuarios = [usuario1, usuario2]

In [80]:
for inx_usuario, usuario in enumerate(usuarios):
    recomendacao = avaliar_usuario(usuario)
    print("Filmes recomendados para o usuário", inx_usuario + 1)
    for i in range(len(usuario[0])):
        if usuario[0][i] == 0 and recomendacao[0][i] == 1:
            print(filmes[i])
    print("\n")

Filmes recomendados para o usuário 1
O chamado


Filmes recomendados para o usuário 2
O chamado
American pie


